In [1]:
import os
import pandas as pd
import numpy as np
import base64
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import os
from collections import defaultdict
import hashlib

In [2]:
curr_dir = os.getcwd()

### Places

In [106]:
data_places = pd.read_csv('data/Yaroslavl_places.csv', sep = ',')
data_places.head()

,XID,Name,Kind,City,OSM,WikiData,Rate,Lon,Lat
0,N552713468,Н.А. Некрасову,"historic,monuments_and_memorials,interesting_p...",Ярославль,node/552713468,Q4343284,3h,39.892933,57.632572
1,W125448768,Российский государственный академический театр...,"cultural,theatres_and_entertainments,interesti...",Ярославль,way/125448768,Q4398052,3h,39.884796,57.627155
2,W239408057,Ярославская соборная мечеть,"religion,other_temples,interesting_places",Ярославль,way/239408057,Q4281966,3h,39.875633,57.636059
3,W44896191,Церковь Спаса на городу,"religion,churches,interesting_places,eastern_o...",Ярославль,way/44896191,Q16715792,3h,39.895622,57.622604
4,W617228287,Спасо-Преображенский монастырь,"religion,monasteries,interesting_places",Ярославль,way/617228287,Q1078595,3h,39.889442,57.621704


In [107]:
data_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   XID       99 non-null     object 
 1   Name      99 non-null     object 
 2   Kind      99 non-null     object 
 3   City      99 non-null     object 
 4   OSM       66 non-null     object 
 5   WikiData  96 non-null     object 
 6   Rate      99 non-null     object 
 7   Lon       99 non-null     float64
 8   Lat       99 non-null     float64
dtypes: float64(2), object(7)
memory usage: 7.1+ KB


XID: Уникальный идентификатор для объекта на карте.

Name: Название объекта.

Kind: Тип объекта, категория объекта.

City: Название города, в котором расположен объект.

OSM: Не понятно что это.

WikiData: Ссылка на соответствующую страницу в базе данных WikiData. WikiData - это свободная база знаний, связанная с проектом Википедии, и содержит информацию о множестве объектов, включая географические места.

Rate: Не понятно что.

Lon: Долгота местоположения объекта.

Lat: Широта местоположения объекта.

In [108]:
data_places.isnull().sum()

XID          0
Name         0
Kind         0
City         0
OSM         33
WikiData     3
Rate         0
Lon          0
Lat          0
dtype: int64

In [109]:
# Show nan values OSM
data_places[pd.isna(data_places['OSM'])].head()

,XID,Name,Kind,City,OSM,WikiData,Rate,Lon,Lat
42,Q19908952,Демидовский сад,"gardens_and_parks,cultural,urban_environment,i...",Ярославль,NaN,Q19908952,3h,39.897240,57.625038
43,Q1189431,Мемориальный дом-музей М. Богдановича,"biographical_museums,museums,cultural,interest...",Ярославль,NaN,Q1189431,3h,39.870735,57.624523
44,Q4110247,Вечный огонь (Ярославль),"historic,monuments_and_memorials,interesting_p...",Ярославль,NaN,Q4110247,3h,39.898975,57.623791
45,Q4151348,Губернаторский дом (Ярославль),"historic_architecture,architecture,interesting...",Ярославль,NaN,Q4151348,3h,39.897221,57.628334
46,Q4538913,Ярославский филиал Военно-космической академии...,"other,unclassified_objects,interesting_places,...",Ярославль,NaN,Q4538913,3h,39.877613,57.607079


In [111]:
# data_places = pd.read_csv('data/Yaroslavl_images.csv', sep = ',')

# filtered = data_places[data_places['Name'].str.contains('[a-zA-Z]')]
filtered = data_places[data_places['Name'].str.contains('Музей-заповедник')]
filtered.head()

,XID,Name,Kind,City,OSM,WikiData,Rate,Lon,Lat


In [112]:
# List of coloumns
print(data_places.columns)

Index(['XID', 'Name', 'Kind', 'City', 'OSM', 'WikiData', 'Rate', 'Lon', 'Lat'], dtype='object')


In [113]:
# Show dublicates
duplicate_places = data_places[data_places.duplicated(subset=['WikiData'], keep=False)]
duplicate_places.sort_values(by='WikiData').head(50)

,XID,Name,Kind,City,OSM,WikiData,Rate,Lon,Lat
4,W617228287,Спасо-Преображенский монастырь,"religion,monasteries,interesting_places",Ярославль,way/617228287,Q1078595,3h,39.889442,57.621704
5,W45013546,Спасо-Преображенский собор,"religion,cathedrals,monasteries,interesting_pl...",Ярославль,way/45013546,Q1078595,3h,39.889511,57.621506
64,W72225050,Леонтьевское кладбище,"cemeteries,historic,burial_places,interesting_...",Ярославль,way/72225050,Q30709938,3,39.834396,57.629993
67,W96530735,Леонтьевская церковь,"cemeteries,religion,other_temples,historic,bur...",Ярославль,way/96530735,Q30709938,3,39.834888,57.630680
60,N3691243291,Музей зарубежного искусства,"cultural,museums,interesting_places,art_galleries",Ярославль,node/3691243291,Q3329597,3,39.895790,57.627102
61,N964688408,Ярославский художественный музей (Губернаторск...,"cultural,museums,interesting_places,art_galleries",Ярославль,node/964688408,Q3329597,3,39.897274,57.628376
24,N4997897721,Знаменская церковь,"towers,religion,architecture,churches,interest...",Ярославль,node/4997897721,Q4113610,3h,39.885262,57.625439
26,W136184765,Власьевская (Знаменская) башня,"towers,religion,architecture,churches,interest...",Ярославль,way/136184765,Q4113610,3h,39.884975,57.625359
16,N1361341279,Кирило-Афанасиевский мужской монастырь,"religion,monasteries,interesting_places",Ярославль,node/1361341279,Q4221355,3h,39.895275,57.625294
20,W462014550,Спасо-Афанасиевский Ярославский монастырь,"religion,monasteries,interesting_places",Ярославль,way/462014550,Q4221355,3h,39.895222,57.625370


In [114]:
# averaged_places = data_places.groupby('WikiData').agg({'Lon': 'mean', 'Lat': 'mean'}).reset_index()

def aggregate_names(names):
    return set(names)

# Агрегация данных с преобразованием столбца Name в список
aggregated_places = data_places.groupby('WikiData').agg({'Name': aggregate_names,
                                                         'Kind': 'first',
                                                         'City': 'first',
                                                         'Rate': 'first',
                                                         'Lon': 'mean',
                                                         'Lat': 'mean'}).reset_index()
# data_places = pd.merge(averaged_places, data_places.drop_duplicates(subset=['WikiData']), on='WikiData')
aggregated_places.head(50)

,WikiData,Name,Kind,City,Rate,Lon,Lat
0,Q104554178,{Церковь Николы в Меленках},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947
1,Q1078595,"{Спасо-Преображенский монастырь, Спасо-Преобра...","religion,monasteries,interesting_places",Ярославль,3h,39.889477,57.621605
2,Q1140551,{Успенский собор},"religion,cathedrals,churches,interesting_place...",Ярославль,3,39.901978,57.622429
3,Q1189431,{Мемориальный дом-музей М. Богдановича},"biographical_museums,museums,cultural,interest...",Ярославль,3h,39.870735,57.624523
4,Q12140265,{Храм Петра и Павла},"religion,churches,interesting_places,eastern_o...",Ярославль,3,39.843700,57.602531
5,Q1365930,{Церковь Михаила Архангела},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.892590,57.622082
6,Q13668162,{Петропавловский парк},"gardens_and_parks,cultural,urban_environment,i...",Ярославль,3h,39.845001,57.599445
7,Q13668166,{Власьевский сквер},"urban_environment,gardens_and_parks,cultural,i...",Ярославль,3,39.882793,57.626457
8,Q13668665,{Храм Ярославской иконы Божией Матери},"religion,churches,interesting_places,eastern_o...",Ярославль,3,39.908268,57.582783
9,Q16715602,{Храм Зосимы и Савватия},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.903271,57.636955


In [115]:
aggregated_places.head(25)

,WikiData,Name,Kind,City,Rate,Lon,Lat
0,Q104554178,{Церковь Николы в Меленках},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947
1,Q1078595,"{Спасо-Преображенский монастырь, Спасо-Преобра...","religion,monasteries,interesting_places",Ярославль,3h,39.889477,57.621605
2,Q1140551,{Успенский собор},"religion,cathedrals,churches,interesting_place...",Ярославль,3,39.901978,57.622429
3,Q1189431,{Мемориальный дом-музей М. Богдановича},"biographical_museums,museums,cultural,interest...",Ярославль,3h,39.870735,57.624523
4,Q12140265,{Храм Петра и Павла},"religion,churches,interesting_places,eastern_o...",Ярославль,3,39.843700,57.602531
5,Q1365930,{Церковь Михаила Архангела},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.892590,57.622082
6,Q13668162,{Петропавловский парк},"gardens_and_parks,cultural,urban_environment,i...",Ярославль,3h,39.845001,57.599445
7,Q13668166,{Власьевский сквер},"urban_environment,gardens_and_parks,cultural,i...",Ярославль,3,39.882793,57.626457
8,Q13668665,{Храм Ярославской иконы Божией Матери},"religion,churches,interesting_places,eastern_o...",Ярославль,3,39.908268,57.582783
9,Q16715602,{Храм Зосимы и Савватия},"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.903271,57.636955


In [116]:
aggregated_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   WikiData  89 non-null     object 
 1   Name      89 non-null     object 
 2   Kind      89 non-null     object 
 3   City      89 non-null     object 
 4   Rate      89 non-null     object 
 5   Lon       89 non-null     float64
 6   Lat       89 non-null     float64
dtypes: float64(2), object(5)
memory usage: 5.0+ KB


In [117]:
def kind_analysis():
    tags = aggregated_places['Kind'].str.split(',')

    # Создание списка всех тегов
    all_tags = [tag.strip() for sublist in tags for tag in sublist]

    # Подсчет частоты встречаемости тегов
    tag_counts = Counter(all_tags)

    # Преобразование результатов в DataFrame
    tag_counts_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count'])

    # Сортировка по убыванию количества встречаемости
    tag_counts_df = tag_counts_df.sort_values(by='Count', ascending=False)

    # Вывод результатов
    print(tag_counts_df)

# kind_analysis()

### Images

In [118]:
data_images = pd.read_csv('data/Yaroslavl_images.csv', sep = ',')
data_images.head()

,name,img
0,Церковь Рождества Христова в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,Церковь Рождества Христова в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,Церковь Рождества Христова в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,Церковь Рождества Христова в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,Церковь Рождества Христова в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...


In [119]:
filtered = data_images[data_images['name'].str.contains('Музей-заповедник')]
filtered.head()

,name,img


In [120]:
data_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    4372 non-null   object
 1   img     4372 non-null   object
dtypes: object(2)
memory usage: 68.4+ KB


In [121]:
data_images.isnull().sum()

name    0
img     0
dtype: int64

In [122]:
data_images['name'].unique().shape

(99,)

In [123]:
aggregated_places_exploded = aggregated_places.explode('Name')
data_images_exploded = data_images.explode('name')

# Объединение таблиц по столбцам Name и name
merged_data = pd.merge(aggregated_places_exploded, data_images_exploded, left_on='Name', right_on='name', how='left')

# Вывод записей, которым не удалось найти пару
unmatched_records = merged_data[merged_data['name'].isnull()]

# Печать результатов
print(unmatched_records)

Empty DataFrame
Columns: [WikiData, Name, Kind, City, Rate, Lon, Lat, name, img]
Index: []


In [84]:
def choose_name(names):
    name_lengths = [len(name) for name in names]
    filtered_names = [name for name in names if '№' not in name]
    if filtered_names:
        return max(filtered_names, key=len)
    else:
        return max(names, key=len)

# aggregated_places_grouped = merged_data.groupby('WikiData')['Name'].first().reset_index()
aggregated_places_grouped = merged_data.groupby('WikiData')['Name'].agg(choose_name).reset_index()

# Объединение таблиц aggregated_places_grouped и data_images по столбцу 'WikiData'
merged_data = pd.merge(aggregated_places_grouped, merged_data, on='WikiData', how='left')

merged_data.rename(columns={'Name_x': 'Name'}, inplace=True)

# Удаление столбца Name_y
merged_data.drop(columns=['Name_y', 'name'], inplace=True)

# Вывод результатов
merged_data

,WikiData,Name,Kind,City,Rate,Lon,Lat,img
0,Q104554178,Церковь Николы в Меленках,"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
1,Q104554178,Церковь Николы в Меленках,"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
2,Q104554178,Церковь Николы в Меленках,"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
3,Q104554178,Церковь Николы в Меленках,"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4,Q104554178,Церковь Николы в Меленках,"religion,churches,interesting_places,eastern_o...",Ярославль,3h,39.834522,57.609947,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
...,...,...,...,...,...,...,...,...
4004,Q97161882,Михайловские курганы,"historic,archaeology,interesting_places,other_...",Ярославль,2h,39.747612,57.765659,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4005,Q97161882,Михайловские курганы,"historic,archaeology,interesting_places,other_...",Ярославль,2h,39.747612,57.765659,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4006,Q97161882,Михайловские курганы,"historic,archaeology,interesting_places,other_...",Ярославль,2h,39.747612,57.765659,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
4007,Q97161882,Михайловские курганы,"historic,archaeology,interesting_places,other_...",Ярославль,2h,39.747612,57.765659,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...


In [22]:
# Show unique values in the column image where WikiData == Q27921892
merged_data.loc[merged_data['WikiData'] == 'Q27921892', 'image'].unique().shape
# merged_data[merged_data['WikiData'] == 'Q27921892']
# merged_data

(45,)

In [24]:
TARGET_COLUMN = 'Name'

output_dir = os.path.join(curr_dir, 'data_by_name')
os.makedirs(output_dir, exist_ok=True)
image_paths = []
forbidden_chars = ['/', '\\', '?', '%', '*', ':', '|', '"', '<', '>', '.']

def replace_forbidden_chars(text):
    for char in forbidden_chars:
        text = text.replace(char, '_')
    return text

for index, row in merged_data.iterrows():
    image_data = base64.b64decode(row['image'])
    class_dir = os.path.join(output_dir, replace_forbidden_chars(row[TARGET_COLUMN]))
    os.makedirs(class_dir, exist_ok=True)
    image_path = os.path.join(class_dir, f"image_{index}.jpg")
    with open(image_path, "wb") as file:
        file.write(image_data)
    image_paths.append(image_path)

merged_data['image_path'] = image_paths

In [33]:
csv_file_path = 'processed_data.csv'
merged_data.to_csv(csv_file_path, index=False)

In [32]:
# Функция для поиска и удаления повторяющихся файлов в папке
def remove_duplicates(folder):
    # Словарь для хранения хешей файлов
    file_hashes = defaultdict(list)
    
    # Просмотр всех файлов в папке
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        
        # Пропускаем папки
        if os.path.isdir(filepath):
            continue
        
        # Вычисление хеша файла
        with open(filepath, "rb") as f:
            file_hash = hashlib.md5(f.read()).hexdigest()
        
        # Добавление хеша в словарь
        file_hashes[file_hash].append(filepath)
    
    # Проверка наличия повторяющихся файлов и удаление их
    for files_list in file_hashes.values():
        if len(files_list) > 1:
            # Оставляем только один файл из списка
            for file_to_remove in files_list[1:]:
                os.remove(file_to_remove)

# Список папок, в которых нужно удалить повторяющиеся изображения
# list of folders in data_by_name
folders_list = os.listdir('data_by_name')

# Применение функции для каждой папки из списка
for folder in folders_list:
    remove_duplicates(os.path.join(curr_dir, 'data_by_name', folder))